# Глава 7: Создание чат-приложений
## Быстрый старт с Github Models API

Этот ноутбук адаптирован из [репозитория примеров Azure OpenAI](https://github.com/Azure/azure-openai-samples?WT.mc_id=academic-105485-koreyst), который содержит ноутбуки для работы с сервисами [Azure OpenAI](notebook-azure-openai.ipynb).


# Обзор  
«Большие языковые модели — это функции, которые преобразуют текст в текст. Получив входную строку текста, большая языковая модель пытается предсказать, какой текст будет следующим»(1). Этот «быстрый старт» познакомит пользователей с основными понятиями LLM, ключевыми требованиями пакета для начала работы с AML, даст простое введение в проектирование подсказок и приведет несколько коротких примеров различных сценариев использования.


## Оглавление  

[Обзор](../../../../07-building-chat-applications/python)  
[Как использовать OpenAI Service](../../../../07-building-chat-applications/python)  
[1. Создание вашего OpenAI Service](../../../../07-building-chat-applications/python)  
[2. Установка](../../../../07-building-chat-applications/python)    
[3. Учетные данные](../../../../07-building-chat-applications/python)  

[Примеры использования](../../../../07-building-chat-applications/python)    
[1. Суммирование текста](../../../../07-building-chat-applications/python)  
[2. Классификация текста](../../../../07-building-chat-applications/python)  
[3. Генерация новых названий продуктов](../../../../07-building-chat-applications/python)  
[4. Тонкая настройка классификатора](../../../../07-building-chat-applications/python)  

[Ссылки](../../../../07-building-chat-applications/python)


### Создайте свой первый промпт  
Это короткое упражнение даст базовое представление о том, как отправлять промпты модели в Github Models для простой задачи "суммирования".


**Шаги**:  
1. Установите библиотеку `azure-ai-inference` в вашу среду Python, если она ещё не установлена.  
2. Загрузите стандартные вспомогательные библиотеки и настройте учетные данные для Github Models.  
3. Выберите модель для вашей задачи  
4. Сформулируйте простой промпт для модели  
5. Отправьте ваш запрос в API модели!


### 1. Установите `azure-ai-inference`


In [ ]:
%pip install azure-ai-inference

In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

token = os.environ["GITHUB_TOKEN"]
endpoint = "https://models.inference.ai.azure.com"

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

### 3. Поиск подходящей модели  
Модели GPT-3.5-turbo или GPT-4 способны понимать и генерировать естественный язык.


In [ ]:
# Select the General Purpose curie model for text
model_name = "gpt-4o"

## 4. Проектирование запросов  

«Волшебство больших языковых моделей заключается в том, что, обучаясь минимизировать ошибку предсказания на огромных объемах текста, модели в итоге осваивают понятия, полезные для этих предсказаний. Например, они осваивают такие вещи, как»(1):

* как правильно писать слова
* как работает грамматика
* как перефразировать
* как отвечать на вопросы
* как вести диалог
* как писать на разных языках
* как программировать
* и многое другое

#### Как управлять большой языковой моделью  
«Из всех входных данных для большой языковой модели самым значимым является текстовый запрос»(1).

Большие языковые модели можно побудить к генерации ответа несколькими способами:

Инструкция: Скажите модели, что вы хотите получить  
Завершение: Побудите модель закончить начатое вами  
Демонстрация: Покажите модели, что вы хотите, с помощью:
Нескольких примеров в самом запросе  
Сотен или тысяч примеров в обучающем датасете для дообучения»



#### Существует три основных правила создания запросов:

**Показывайте и объясняйте**. Дайте понять, что вы хотите, с помощью инструкций, примеров или их сочетания. Если вы хотите, чтобы модель отсортировала список по алфавиту или определила тональность абзаца, покажите ей, что именно это вы ожидаете.

**Используйте качественные данные**. Если вы хотите создать классификатор или чтобы модель следовала определенному шаблону, убедитесь, что у вас достаточно примеров. Обязательно проверьте свои примеры на ошибки — модель обычно достаточно умна, чтобы игнорировать простые опечатки и все равно дать ответ, но она может решить, что это сделано специально, и это повлияет на результат.

**Проверьте настройки.** Параметры temperature и top_p определяют, насколько детерминированно модель будет генерировать ответ. Если вы ожидаете единственно правильный ответ, эти параметры стоит понизить. Если же вам нужны более разнообразные ответы, их можно повысить. Самая частая ошибка — думать, что эти параметры отвечают за «ум» или «креативность» модели.


Источник: https://learn.microsoft.com/azure/ai-services/openai/overview


In [ ]:
# Create your first prompt
text_prompt = "Should oxford commas always be used?"

response = client.complete(
  model=model_name,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":text_prompt},])

response.choices[0].message.content

In [ ]:

response = client.complete(
  model=model_name,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":text_prompt},])

response.choices[0].message.content

## Суммирование текста  
#### Задача  
Суммируйте текст, добавив 'tl;dr:' в конце отрывка. Обратите внимание, что модель понимает, как выполнять множество задач без дополнительных инструкций. Вы можете поэкспериментировать с более описательными подсказками, чем tl;dr, чтобы изменить поведение модели и настроить получаемое вами суммирование(3).  

Недавние исследования показали значительный прогресс во многих задачах и бенчмарках обработки естественного языка благодаря предварительному обучению на большом корпусе текстов с последующей донастройкой под конкретную задачу. Хотя архитектура обычно не зависит от задачи, этот метод всё равно требует специализированных датасетов для дообучения, содержащих тысячи или десятки тысяч примеров. В отличие от этого, люди обычно могут выполнять новую языковую задачу, имея всего несколько примеров или простые инструкции — с чем современные системы обработки языка всё ещё в основном не справляются. Здесь мы показываем, что увеличение масштабов языковых моделей значительно улучшает универсальную работу в режиме few-shot, иногда даже достигая уровня конкуренции с предыдущими лучшими подходами дообучения.

Tl;dr


# Упражнения для различных сценариев  
1. Суммировать текст  
2. Классифицировать текст  
3. Придумывать новые названия продуктов


In [ ]:
prompt = "Recent work has demonstrated substantial gains on many NLP tasks and benchmarks by pre-training on a large corpus of text followed by fine-tuning on a specific task. While typically task-agnostic in architecture, this method still requires task-specific fine-tuning datasets of thousands or tens of thousands of examples. By contrast, humans can generally perform a new language task from only a few examples or from simple instructions - something that current NLP systems still largely struggle to do. Here we show that scaling up language models greatly improves task-agnostic, few-shot performance, sometimes even reaching competitiveness with prior state-of-the-art fine-tuning approaches.\n\nTl;dr"


In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.complete(
  model=model_name,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt},])

response.choices[0].message.content

## Классификация текста  
#### Задача  
Классифицируйте элементы по категориям, которые задаются во время выполнения. В следующем примере мы указываем как категории, так и текст для классификации в подсказке (*playground_reference).

Запрос клиента: Здравствуйте, одна из клавиш на моей клавиатуре ноутбука недавно сломалась, и мне нужен заменитель:

Классифицированная категория:


In [ ]:
prompt = "Classify the following inquiry into one of the following: categories: [Pricing, Hardware Support, Software Support]\n\ninquiry: Hello, one of the keys on my laptop keyboard broke recently and I'll need a replacement:\n\nClassified category:"
print(prompt)

In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.complete(
  model=model_name,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt},])

response.choices[0].message.content

## Генерация новых названий продуктов
#### Задача
Придумайте названия продуктов на основе примеров слов. В этом задании мы указываем информацию о продукте, для которого нужно придумать название. Также приводим похожий пример, чтобы показать желаемый шаблон. Мы установили высокое значение температуры, чтобы повысить случайность и получить более креативные варианты.

Описание продукта: Домашний аппарат для приготовления молочных коктейлей
Ключевые слова: быстро, полезно, компактно.
Названия продуктов: HomeShaker, Fit Shaker, QuickShake, Shake Maker

Описание продукта: Пара обуви, которая подходит на любую ногу.
Ключевые слова: адаптивный, подходит, omni-fit.


In [ ]:
prompt = "Product description: A home milkshake maker\nSeed words: fast, healthy, compact.\nProduct names: HomeShaker, Fit Shaker, QuickShake, Shake Maker\n\nProduct description: A pair of shoes that can fit any foot size.\nSeed words: adaptable, fit, omni-fit."

print(prompt)

In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.complete(
  model=model_name,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt}])

response.choices[0].message.content

# Ссылки  
- [Openai Cookbook](https://github.com/openai/openai-cookbook?WT.mc_id=academic-105485-koreyst)  
- [Примеры OpenAI Studio](https://oai.azure.com/portal?WT.mc_id=academic-105485-koreyst)  
- [Лучшие практики дообучения GPT-3 для классификации текста](https://docs.google.com/document/d/1rqj7dkuvl7Byd5KQPUJRxc19BJt8wo0yHNwK84KfU3Q/edit#?WT.mc_id=academic-105485-koreyst)


# Для дополнительной помощи  
[OpenAI Commercialization Team](AzureOpenAITeam@microsoft.com)


# Участники
* [Chew-Yean Yam](https://www.linkedin.com/in/cyyam/)



---

**Отказ от ответственности**:  
Этот документ был переведен с помощью сервиса автоматического перевода [Co-op Translator](https://github.com/Azure/co-op-translator). Несмотря на наши усилия обеспечить точность, автоматические переводы могут содержать ошибки или неточности. Оригинальный документ на исходном языке следует считать авторитетным источником. Для получения критически важной информации рекомендуется профессиональный перевод человеком. Мы не несем ответственности за любые недоразумения или неправильные толкования, возникшие в результате использования данного перевода.
